In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('YoutubeCommentsDataSet.csv')
df.head()

,Comment,Sentiment
0,lets not forget that apple pay in 2014 require...,neutral
1,here in nz 50 of retailers don’t even have con...,negative
2,i will forever acknowledge this channel with t...,positive
3,whenever i go to a place that doesn’t take app...,negative
4,apple pay is so convenient secure and easy to ...,positive


In [3]:
df.Comment

0        lets not forget that apple pay in 2014 require...
1        here in nz 50 of retailers don’t even have con...
2        i will forever acknowledge this channel with t...
3        whenever i go to a place that doesn’t take app...
4        apple pay is so convenient secure and easy to ...
                               ...                        
18403    i really like the point about engineering tool...
18404    i’ve just started exploring this field and thi...
18405    excelente video con una pregunta filosófica pr...
18406    hey daniel just discovered your channel a coup...
18407    this is great focus is key a playful approach ...
Name: Comment, Length: 18408, dtype: object

In [4]:
df.info()
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18408 entries, 0 to 18407
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Comment    18364 non-null  object
 1   Sentiment  18408 non-null  object
dtypes: object(2)
memory usage: 287.8+ KB
Comment      44
Sentiment     0
dtype: int64


In [5]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18364 entries, 0 to 18407
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Comment    18364 non-null  object
 1   Sentiment  18364 non-null  object
dtypes: object(2)
memory usage: 430.4+ KB


In [6]:
df = df.rename(columns={"Comment": "yorum", "Sentiment": "sonuc"})
df.head()

,yorum,sonuc
0,lets not forget that apple pay in 2014 require...,neutral
1,here in nz 50 of retailers don’t even have con...,negative
2,i will forever acknowledge this channel with t...,positive
3,whenever i go to a place that doesn’t take app...,negative
4,apple pay is so convenient secure and easy to ...,positive


In [7]:
df['sonuc'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
stop_words = set(stopwords.words('english'))
label_encoder = LabelEncoder()
df['sonuc'] = label_encoder.fit_transform(df['sonuc'])
for comment in stop_words:
    comment = " "+comment+" "
    df['yorum'] = df['yorum'].str.replace(comment," ")

cv = CountVectorizer(max_features = 50,stop_words=list(stop_words))
x = cv.fit_transform(df['yorum']).toarray()
y = df['sonuc'].values
x=x.astype('float64')
y = y.astype('float64')
y= to_categorical(y, num_classes=3)

In [9]:
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [10]:
y

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [11]:
print(x.dtype)
print(y.dtype)

float64
float32


In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.80,random_state=42)

In [13]:
print(x_train.shape)
print(y_train.shape)

(14691, 50)
(14691, 3)


In [14]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(256, activation='relu', input_dim=50))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train,y_train,epochs=256,batch_size=32,validation_split=0.24)

Epoch 1/256
349/349 [==============================] - 1s 3ms/step - loss: 0.7912 - accuracy: 0.6282 - val_loss: 0.7558 - val_accuracy: 0.6461
Epoch 2/256
349/349 [==============================] - 1s 2ms/step - loss: 0.7480 - accuracy: 0.6491 - val_loss: 0.7485 - val_accuracy: 0.6520
Epoch 3/256
349/349 [==============================] - 1s 2ms/step - loss: 0.7319 - accuracy: 0.6522 - val_loss: 0.7471 - val_accuracy: 0.6563
Epoch 4/256
349/349 [==============================] - 1s 2ms/step - loss: 0.7152 - accuracy: 0.6600 - val_loss: 0.7638 - val_accuracy: 0.6551
Epoch 5/256
349/349 [==============================] - 1s 2ms/step - loss: 0.6999 - accuracy: 0.6633 - val_loss: 0.7650 - val_accuracy: 0.6378
Epoch 6/256
349/349 [==============================] - 1s 2ms/step - loss: 0.6848 - accuracy: 0.6721 - val_loss: 0.7847 - val_accuracy: 0.6523
Epoch 7/256
349/349 [==============================] - 1s 2ms/step - loss: 0.6665 - accuracy: 0.6787 - val_loss: 0.8168 - val_accuracy: 0.6256

In [15]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

115/115 [==============================] - 0s 3ms/step - loss: 2.7707 - accuracy: 0.6256
Test Loss: 2.770742416381836
Test Accuracy: 0.6256465911865234


In [17]:
class_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Sınıf Eşleşmeleri:", class_mapping)

Sınıf Eşleşmeleri: {'negative': 0, 'neutral': 1, 'positive': 2}


In [25]:
df.yorum[25]

'dan man saving day riley needs give theme like brian electrician'

In [28]:
df.sonuc[25]

2

In [30]:
new_comment = ["This product is amazing!", "I did not like this product at all.","dan man saving day riley needs give theme like brian electrician"]
new_comment_vec = cv.transform(new_comment).toarray()
predictions = model.predict(new_comment_vec)

predicted_classes = np.argmax(predictions, axis=1)
print("Tahmin Edilen Sınıflar:", predicted_classes)

1/1 [==============================] - 0s 14ms/step
Tahmin Edilen Sınıflar: [2 1 2]
